## Deep Learning
Assignment 5
The goal of this assignment is to train a Word2Vec skip-gram model over Text8 data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

/Users/wangwei/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Download the data from the source website 

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """download  file if not present, and make sure it's the right size"""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url+ filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s'% filename)
    else:
        print(statinfo.st_size)
        raise Exception(
         'Failed to verify' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


## Read the data into a string.


In [3]:
# f.read(text8) : Return the bytes of the file name in the archive. 
# name is the name of the file in the archive,
# f.namelist()[0] : text8
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


## Build the dictionary and replace rare words with UNK token.


In [4]:
# We have all the 'words', let's build the 'dictionary'.
# dictonary first only select the most frequently occuring 5000 words
# dictionary also rate these words by their order, 
# e.g., the most frequent word is given value 1, less is 2, etc.
# output data is a mapping of 'words' into 'numbers'.
vocabulary_size = 50000
def build_dataset(words):
    # build count : a list can maintain its order
    count = [['UNK', -1]]
    # 括号里是一个list,extend是一个循环的添加list里的每个元素
    count.extend(collections.Counter(words).most_common(
                                              vocabulary_size - 1))

    # build dictionary : by the order of 'count'
    # dictionary keeps track of the frequency ranking of the word
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)

    # build data （a list ）: mapping each word into an index
    data = list()
    unk_count = 0
    for word in words:
        #如果这个word 是头5000频繁使用的，index是它的frequency rank
        if word in dictionary:
            index = dictionary[word]
        #反之，这个词是不频繁词，index 设置为0，并且给unk_count 加1
        else:
            index = 0 # dictionary['UNK']
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    # 最后，方便后续处理，revserse_dictionary 是以 frequency rank 为 key，word 为value
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('dictionary words: %s counts: %d'% (dictionary.keys()[0], 
                        dictionary[dictionary.keys()[0]]))
print('most common words (+ UNK)', count[:5])
print('Sample data', data[:10])
print('data length %i is the same as words %i' % 
                    (len(data), len(words)))
del words

dictionary words: fawn counts: 45848
most common words (+ UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156]
data length 17005207 is the same as words 17005207


## Function to generate a training batch for the skip-gram model.


In [5]:
data_index =0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    
    # batch_size 应该是 num_skips 的整数倍, e.g. 8
    assert batch_size % num_skips == 0
    # num_skips 的长度小于 skip_window的2倍, e.g., numskip =2, skipwindow =1
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype = np.int32)
    labels = np.ndarray(shape=(batch_size,1), dtype = np.int32)
    span = 2* skip_window +1 #[skip window target skip window]
    # deque : list-like container with fast appends and pops on either end
    buffer = collections.deque(maxlen = span)
    
    # 把长度是5的数据放到buffer里
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    


    # 对于一个长度是8的batch，numskip为2的情况，要循环4次
    for i in range(batch_size // num_skips):
        # 因为长度是3，window是1，刚好是中间的位置
        target = skip_window # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        
        # num_skips = 2
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i *  num_skips + j] = buffer[skip_window]
            labels[i *num_skips + j,0] = buffer[target]
        
        
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    print('data in buffer are:', [reverse_dictionary[di] for di in buffer])    
    return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])



data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']
data in buffer are: ['term', 'of', 'abuse']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['anarchism', 'as', 'originated', 'a', 'as', 'term', 'of', 'a']
data in buffer are: ['as', 'a', 'term', 'of', 'abuse']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['originated', 'term', 'a', 'anarchism', 'of', 'as', 'term', 'originated']


In [6]:
batch_size = 128
embedding_size = 128 # dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):
    
    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size,1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # Variables.
    embeddings = tf.Variable(
     tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    print('embeddings:', embeddings.get_shape().as_list())
    softmax_weights = tf.Variable(
     tf.truncated_normal([vocabulary_size, embedding_size], stddev = 1.0/math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    # model.
    # Look up embeddings for inputs
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    print('embed:', embed.get_shape().as_list())
    # compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed, train_labels, num_sampled, vocabulary_size))
    
    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
    
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims = True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
       normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

embeddings: [50000, 128]
embed: [128, 128]


In [ ]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
    
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
    final_embeddings = normalized_embeddings.eval()


Initialized
data in buffer are: ['derived', 'from', 'the']
Average loss at step 0: 8.142462
Nearest to were: tulare, blaming, cycladic, pater, characterizes, livery, med, gabal,
Nearest to into: abdullah, epinephrine, devon, classicism, domitian, voltages, eloquence, bhfuil,
Nearest to his: wah, plumes, outbreaks, asparagales, warne, barkley, macphail, clinicians,
Nearest to most: void, atiyah, ligny, bougainville, pieced, agostino, uncomplicated, hastily,
Nearest to may: overridden, cursus, brasses, easter, universidad, stopper, codons, telephone,
Nearest to over: chunks, conceit, creep, yasser, quantifiers, draws, trang, enchantment,
Nearest to united: boromir, welcoming, deficits, sufficiency, discredited, false, footballer, webs,
Nearest to a: albertosaurus, tegmark, mortally, sleeps, ruin, oneself, tough, psychic,
Nearest to state: trisomy, gpl, kamal, kaltenbrunner, troposphere, belgrade, backwardness, faux,
Nearest to have: journeyed, palmer, lined, isotopic, oecs, passover, gen

In [83]:
help(tf.nn.embedding_lookup)

Help on function embedding_lookup in module tensorflow.python.ops.embedding_ops:

embedding_lookup(params, ids, partition_strategy='mod', name=None, validate_indices=True)
    Looks up `ids` in a list of embedding tensors.
    
    This function is used to perform parallel lookups on the list of
    tensors in `params`.  It is a generalization of
    [`tf.gather()`](../../api_docs/python/array_ops.md#gather), where `params` is
    interpreted as a partition of a larger embedding tensor.
    
    If `len(params) > 1`, each element `id` of `ids` is partitioned between
    the elements of `params` according to the `partition_strategy`.
    In all strategies, if the id space does not evenly divide the number of
    partitions, each of the first `(max_id + 1) % len(params)` partitions will
    be assigned one more id.
    
    If `partition_strategy` is `"mod"`, we assign each id to partition
    `p = id % len(params)`. For instance,
    13 ids are split across 5 partitions as:
    `[[0, 5,